# Recurrent Neural Networks

* so far our vanilla NNs expected a fixed-length input and predicted a fixed-length output (e.g., sentiment classification example, animacy detection)

* we are now looking at neural networks that **handle variable length input**


* the exciting idea behind **Recurrent Neural Networks** (RNN) is that they allows us to work on **sequences** of input, output, or both

What do we mean by a *variable length input*?

A variable length input is a sequence where each input $x$ has a different length.

For instance, the first training instance has $l$ dimensions (say, $l$ tokens), the second input sequence has $m$ dimensions (e.g., tokens).

Mathematically, inspired by the notation of Cho (2015):
$$ \mathbf{x_1} = \langle x_1, ..., x_l \rangle$$ 
$$ \mathbf{x_2} = \langle x_1, ..., x_m \rangle$$
where $l\neq m$


### Detour: very simple example

From Cho (2015). Assume we have a vector $\mathbf{x}$ containing zeros and ones. We want to count the number of 1s.



In [ ]:
def add1(el,s):
    if el==1: return s+1
    else: return s

In [ ]:
v=[0,1,0,0,1,1]
s=0
for el in v:
    s=add1(el,s)
print("count(1):", s)

Two important components:
* memory $s$
* function `add1` is applied to each symbol in input *one at a time* together with memory $s$

$\rightarrow$ input of any length

### Sequences

In language technology we often work with sequences, e.g. sequences of words or sentences, e.g., $$ \mathbf{x_2} = \langle x_1, ..., x_m \rangle$$

#### Approach 1: We have already seen one approach to handle such sequences

taking simply the **mean** of all word vectors in a sentence (e.g., CBOW, see Goldberg's primer); but what about word order?

#### Approach 2: Recurrent neural networks (RNNs)

### RNNs (Elman, 1990): Dependence on previous step

* RNNs are called **recurrent** because they predict the next output being dependent on the previous output (i.e., like having a *memory* of what has been seen so far)


More formally, following the notation in Goldberg (2015):

* $\mathbf{x_{1:n}}$ input sequence
* $\mathbf{s_0}$ starting state (inital state)
* function $R$ ("memory up so far") that takes a state vector $\mathbf{s_i}$ and input vector $x_i$ and produces a new state $ \mathbf{s_{i+1}}$
* function $O$ maps from state to output $\mathbf{y}$

Formulation of an RNN (Goldberg 2015): <img src="pics/rnn0.png" width=400> 

Graphical representation of an RNN (Goldberg 2015): <img src="pics/rnn1.png" width=400> 

### Unrolling over time

<img src="pics/rnn2.png">

Note: $\theta$ shared parameters over time!

##### Expansion at state 4: 
<img src="pics/rnn3.png">
Note $s_i$ based on all $s_0,..,s_i$.

##### Another visualization (Le Cun et al, 2015)

<img src="http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/09/rnn.jpg" alt="illustration from WildML">
A recurrent neural network and the unfolding in time of the computation involved in its forward computation (LeCun et al., 2015).

However, basic RNN tend to not work well past a few recent time steps (vanishing or exploding gradients; one trick: gradient clipping for exploding gradients; otherwise: alternative models)

### Different formulations of R,O

lead to different instantiations of RNNS:

* LSTM (Long Short-Term Memory) (Hochreiter and Schmidhuber, 1997)
* GRU (Gated Recurrent Units) (Cho et al., 2014)


## Long Short Term Memory network (LSTM)

<img src="https://jhui.github.io/assets/rnn/lstm.png"> 

$h_t$ in RNN serves 2 purpose:

- Make an output prediction, and
- A hidden state representing the data sequence processed so far.


LSTM splits these 2 roles into 2 separate variables $h_t$ and $C$. The hidden state of the LSTM cell is now $C$.


There are 3 gates controlling what information will pass through:

- gate forget controls what part of the previous cell state will be kept.
- gate input controls what part of the new computed information will be added to the cell state C.
- gate out controls what part of the cell state will exposed as the hidden state.


Helps to keep information longer.


* excellent introduction to LSTMS: [http://colah.github.io/posts/2015-08-Understanding-LSTMs/](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
* [article on dropout in RNNs](https://arxiv.org/abs/1409.2329)
* [another post](https://jhui.github.io/2017/03/15/RNN-LSTM-GRU/)

### Example: Classification with an LSTM

Predicting a class label from the last LSTM state (sometimes also known as LSTM **acceptor**, cf. Goldberg, 2015):
<img src="pics/many2one.png">



## LSTM in `DyNet`

In [ ]:
import dynet as dy
pc = dy.ParameterCollection()
NUM_LAYERS=2
INPUT_DIM=50
HIDDEN_DIM=10
builder = dy.LSTMBuilder(NUM_LAYERS, INPUT_DIM, HIDDEN_DIM, pc)

In [ ]:
s0 = builder.initial_state() # get initial state
x1 = dy.vecInput(50)

In [ ]:
s1=s0.add_input(x1)
y1 = s1.output()
# here, we add x1 to the RNN, and the output we get from the top is y (a HIDEN_DIM-dim vector)

In [ ]:
y1.npvalue().shape


In [ ]:
s2=s1.add_input(x1) # we can add another input
y2=s2.output() # call the output of this state

#### Aside: memory efficient transduction

In [ ]:

### more efficient - transduce
dy.renew_cg()
word_embs = [dy.lookup(W_emb, x) for x in words]
fwd_lstm = fwdLSTM.initial_state()

fwd_lstm.init() # get initial state

h_lstm = fwd_lstm.transduce(word_embs) # run through the entire sentence
   
W_sm_exp = dy.parameter(W_sm)
b_sm_exp = dy.parameter(b_sm)
score = W_sm_exp * h_lstm + b_sm_exp


### RNNs:  **sequences** of input, output, or both

Karpathy's illustration of RNNs:
<img src="http://benjaminbolte.com/resources/attention_rnn/karpathy_rnn.jpeg">

* From left to right: (1) Vanilla mode of processing without RNN, from fixed-sized input to fixed-sized output (e.g. image classification). (2) Sequence output (e.g. image captioning takes an image and outputs a sentence of words). (3) Sequence input (e.g. sentiment analysis where a given sentence is classified as expressing positive or negative sentiment). (4) Sequence input and sequence output (e.g. Machine Translation: an RNN reads a sentence in English and then outputs a sentence in French). (5) Synced sequence input and output (e.g. video classification where we wish to label each frame of the video). Notice that in every case are no pre-specified constraints on the lengths sequences because the recurrent transformation (green) is fixed and can be applied as many times as we like.*

##### One to many: Image caption generation, Karpathy and Li (2014)
<img src="pics/karpathy-li-2014.png" width=500>
http://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Karpathy_Deep_Visual-Semantic_Alignments_2015_CVPR_paper.pdf

##### many to many: POS tagging

bidirectional RNN (biRNN/biLSTM) - (Plank et al., 2016):

<img src="pics/bilstm.png">

In [5]:
## bilstm tagger in DyNet - from https://github.com/clab/dynet/blob/master/examples/tagger/bilstmtagger.py

import dynet as dy
from collections import Counter
import random

# format of files: each line is "word<TAB>tag<newline>", blank line is new sentence.
train_file="/Users/bplank/Dropbox/corpora/pos/ud1.3/orgtok/goldpos/da-ud-train.conllu"
test_file="/Users/bplank/Dropbox/corpora/pos/ud1.3/orgtok/goldpos/da-ud-dev.conllu"

class Vocab:
    def __init__(self, w2i):
        self.w2i = dict(w2i)
        self.i2w = {i:w for w,i in w2i.items()}

    @classmethod
    def from_corpus(cls, corpus):
        w2i = {}
        for sent in corpus:
            for word in sent:
                w2i.setdefault(word, len(w2i))
        return Vocab(w2i)

    def size(self):
        return len(self.w2i.keys())

MLP=True

def read(fname):
    sent = []
    for line in open(fname):
        line = line.strip().split()
        if not line:
            if sent: yield sent
            sent = []
        else:
            w,p = line
            sent.append((w,p))

train=list(read(train_file))
test=list(read(test_file))
words=[]
tags=[]
wc=Counter()
for s in train:
    for w,p in s:
        words.append(w)
        tags.append(p)
        wc[w]+=1
words.append("_UNK_")
#words=[w if wc[w] > 1 else "_UNK_" for w in words]
tags.append("_START_")

for s in test:
    for w,p in s:
        words.append(w)

vw = Vocab.from_corpus([words])
vt = Vocab.from_corpus([tags])
UNK = vw.w2i["_UNK_"]

nwords = vw.size()
ntags  = vt.size()

model = dy.Model()
trainer = dy.SimpleSGDTrainer(model)

E = model.add_lookup_parameters((nwords, 128))
p_t1  = model.add_lookup_parameters((ntags, 30))
if MLP:
    pH = model.add_parameters((32, 50*2))
    pO = model.add_parameters((ntags, 32))
else:
    pO = model.add_parameters((ntags, 50*2))

builders=[
        dy.LSTMBuilder(1, 128, 50, model),
        dy.LSTMBuilder(1, 128, 50, model),
        ]

def build_tagging_graph(words, tags, builders):
    dy.renew_cg()
    f_init, b_init = [b.initial_state() for b in builders]

    wembs = [E[w] for w in words]
    wembs = [dy.noise(we,0.1) for we in wembs]

    fw = [x.output() for x in f_init.add_inputs(wembs)]
    bw = [x.output() for x in b_init.add_inputs(reversed(wembs))]

    if MLP:
        H = dy.parameter(pH)
        O = dy.parameter(pO)
    else:
        O = dy.parameter(pO)
    errs = []
    for f,b,t in zip(fw, reversed(bw), tags):
        f_b = dy.concatenate([f,b])
        if MLP:
            r_t = O*(dy.tanh(H * f_b))
        else:
            r_t = O * f_b
        err = dy.pickneglogsoftmax(r_t, t)
        errs.append(err)
    return dy.esum(errs)

def tag_sent(sent, builders):
    dy.renew_cg()
    f_init, b_init = [b.initial_state() for b in builders]
    wembs = [E[vw.w2i.get(w, UNK)] for w,t in sent]

    fw = [x.output() for x in f_init.add_inputs(wembs)]
    bw = [x.output() for x in b_init.add_inputs(reversed(wembs))]

    if MLP:
        H = dy.parameter(pH)
        O = dy.parameter(pO)
    else:
        O = dy.parameter(pO)
    tags=[]
    for f,b,(w,t) in zip(fw,reversed(bw),sent):
        if MLP:
            r_t = O*(dy.tanh(H * dy.concatenate([f,b])))
        else:
            r_t = O*dy.concatenate([f,b])
        out = dy.softmax(r_t)
        chosen = np.argmax(out.npvalue())
        tags.append(vt.i2w[chosen])
    return tags


tagged = loss = 0
for ITER in range(50):
    random.shuffle(train)
    for i,s in enumerate(train,1):
        if i % 5000 == 0:
            trainer.status()
            print(loss / tagged)
            loss = 0
            tagged = 0
        if i % 10000 == 0:
            good = bad = 0.0
            for sent in test:
                tags = tag_sent(sent, builders)
                golds = [t for w,t in sent]
                for go,gu in zip(golds,tags):
                    if go == gu: good +=1 
                    else: bad+=1
            print(good/(good+bad))
        ws = [vw.w2i.get(w, UNK) for w,p in s]
        ps = [vt.w2i[p] for w,p in s]
        sum_errs = build_tagging_graph(ws,ps,builders)
        squared = -sum_errs# * sum_errs
        loss += sum_errs.scalar_value()
        tagged += len(ps)
        sum_errs.backward()
        trainer.update()



KeyboardInterrupt: 

Exercise time! 

## References

* [Kyunghyun Cho's excellent lecture notes, chapter 4](http://arxiv.org/abs/1511.07916)
* [Karpathy's blog on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* [WildMl blog](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/)
    